In [7]:

import numpy as np
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import load_model
import resnet
import keras

data_path_l ='/Data/training_left'
data_path_r ='/Data/training_right'


num_classes=8
batch_size=16
epochs=200
train_images=274


img_channels=3
img_rows=224
img_cols=224


input_imgen = ImageDataGenerator()

def generate_generator_multiple(generator,dir1, dir2,  batch_size):
    genX1 = generator.flow_from_directory(directory=dir1,target_size=(img_rows,img_cols),
                                          color_mode="rgb",
                                          batch_size=batch_size,
                                          class_mode="categorical",
                                          shuffle=True, seed=42,)
                                          
      
    genX2 = generator.flow_from_directory(directory=dir2,target_size=(img_rows,img_cols),
                                          color_mode="rgb",
                                          batch_size=batch_size,
                                          class_mode="categorical",
                                          shuffle=True, seed=42,)
    
    
    while True:
            X1i = genX1.next()
            X2i = genX2.next()

            yield [X1i[0],X2i[0]], [X1i[1],X2i[1]]  #Yield both images and their mutual label
            
             
 
           
inputgenerator=generate_generator_multiple(generator=input_imgen,
                                           dir1=data_path_l,
                                           dir2=data_path_r,
                                           batch_size=batch_size)


def multi_loss_model():
    model_rgb  = resnet.ResnetBuilder.build_resnet_50((img_channels, img_rows, img_cols), num_classes)
    model_disp = resnet.ResnetBuilder.build_resnet_50((img_channels, img_rows, img_cols), num_classes)
    custom_resnet_model = Model(inputs=[model_rgb.input,model_disp.input], outputs=[model_rgb.output,model_disp.output])

    return custom_resnet_model




model_resnet_multi_class= multi_loss_model()
model_resnet_multi_class.compile(optimizer=keras.optimizers.Adam(),
                                 loss=[keras.losses.CategoricalCrossentropy(), keras.losses.CategoricalCrossentropy()],
                                  loss_weights=[0.5, 0.5],
                                 metrics=['accuracy'])
    


history=model_resnet_multi_class.fit_generator(inputgenerator,
                       steps_per_epoch= train_images // batch_size,
                        epochs = epochs)

Epoch 1/200
Found 424 images belonging to 10 classes.
Found 424 images belonging to 10 classes.
 8/26 [========>.....................] - ETA: 4:52 - loss: 11.7381 - dense_3_loss: 1.1969 - dense_4_loss: 1.2206 - dense_3_accuracy: 0.6250 - dense_4_accuracy: 0.6094

KeyboardInterrupt: ignored